In [110]:
import random
import time
import warnings
import sys
import argparse
import shutil
import os.path as osp

from sklearn.model_selection import *
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.optim import SGD
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
import torchvision.transforms as T
import torch.nn.functional as F
from collections import OrderedDict as OD
from torchvision.datasets import *
sys.path.append('../../..')
from dalib.modules.domain_discriminator import DomainDiscriminator
from dalib.adaptation.dann import DomainAdversarialLoss, ImageClassifier
import common.vision.datasets as datasets
import common.vision.models as models
from common.vision.transforms import ResizeImage
from common.utils.data import ForeverDataIterator
from common.utils.metric import accuracy, ConfusionMatrix
from common.utils.meter import AverageMeter, ProgressMeter
from common.utils.logger import CompleteLogger
from common.utils.analysis import collect_feature, tsne, a_distance

from torch.utils.tensorboard import SummaryWriter


In [10]:

# Data loading code
normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

Resize = ResizeImage(256)
Crop = T.RandomResizedCrop(224)

train_transform = T.Compose([   
Resize,
T.CenterCrop(224),
T.RandomHorizontalFlip(),
T.ToTensor(),
normalize
])

val_transform = T.Compose([
    Resize,
    T.CenterCrop(224),
    T.ToTensor(),
    normalize
])

In [41]:
# root = "/Volumes/MacData/2022coding_files/datasets/office31/"
# source = "A" 
# target = "D"
 
# dataset = datasets.__dict__["Office31"]
# train_source_dataset = dataset(root=root, task=source, download=True, transform=train_transform)
# train_target_dataset = dataset(root=root, task=target, download=True, transform=train_transform)

# # Combine the datasets into one
# train_dev_sets = torch.utils.data.ConcatDataset([train_source_dataset, train_target_dataset])

In [118]:
root = r"/Volumes/MacData/2022coding_files/datasets/office31_A2D"
dataset = ImageFolder(root, transform = train_transform)
dataset
kf = KFold(n_splits=3)
spliters = kf.split(dataset)


In [121]:
batch_size = 32
for fold,(train_idx,val_idx) in enumerate(spliters):
    print('------------fold no---------{}----------------------'.format(fold))

    train_dataset = torch.utils.data.Subset(dataset, train_idx)
    valid_dataset = torch.utils.data.Subset(dataset, val_idx)


    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle= True,
        num_workers=2, pin_memory=True)
    
    print(train_loader)

    val_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=batch_size, shuffle=False,
    num_workers=2, pin_memory=True)
    print(val_loader)



0
tensor([[[2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         ...,
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489]],

        [[2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         ...,
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286]],

        [[2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
         [2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
         [2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 

KeyboardInterrupt: 